In [8]:
# install necessary packages finta, gym, jupyterlab, matplotlib, mplfinance, numpy, pandas, quantstats, stable_baselines3, torch
# %pip install finta gym jupyterlab matplotlib mplfinance numpy pandas quantstats stable_baselines3 torch


In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import datetime as dt
import mplfinance 
from renkodf import Renko
from scipy.signal import lfilter
import sys
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
import fx # personal functions
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/")
from meta.env_fx_trading.env_fx import tgym
import finta

In [9]:
tickstory_filename = "C:/Users/WilliamFetzner/Documents/Trading/3mo_EURUSD.csv"
df = pd.read_csv(tickstory_filename)#, header=None, names=['Timestamp','Bid price','Ask price','Bid volume','Ask volume'])
df.head(10)

,Timestamp,Bid price,Ask price,Bid volume,Ask volume
0,20231113 19:00:00:022,1.07013,1.07015,0.12,0.9
1,20231113 19:00:00:229,1.07014,1.07016,1.89,0.9
2,20231113 19:00:00:762,1.07016,1.07018,0.12,4.5
3,20231113 19:00:00:867,1.07015,1.07018,5.40,0.9
4,20231113 19:00:00:970,1.07016,1.07019,4.59,0.9
5,20231113 19:00:04:314,1.07016,1.07020,3.69,0.9
6,20231113 19:00:06:243,1.07016,1.07019,0.12,3.6
7,20231113 19:00:07:285,1.07016,1.07020,3.69,0.9
8,20231113 19:00:07:387,1.07016,1.07019,0.12,3.6
9,20231113 19:00:08:849,1.07016,1.07020,3.69,0.9


In [10]:
# convert date to datetime
df['datetime'] = pd.to_datetime(df['Timestamp'], format='%Y%m%d %H:%M:%S:%f')
# rename bid price to close
df.rename(columns={'Bid price':'close'}, inplace=True)
df_ready = df.set_index('datetime')
# adjust the datetime 7 hrs ahead to match market time
df_ready.index = df_ready.index + pd.Timedelta(hours=7)

In [11]:
ohlc = df_ready[['close']]
ohlc

,close
datetime,
2023-11-14 02:00:00.022,1.07013
2023-11-14 02:00:00.229,1.07014
2023-11-14 02:00:00.762,1.07016
2023-11-14 02:00:00.867,1.07015
2023-11-14 02:00:00.970,1.07016
...,...
2024-02-14 01:59:44.162,1.07083
2024-02-14 01:59:48.793,1.07083
2024-02-14 01:59:49.098,1.07083


In [17]:
brick_size = 0.0003
# psar variables
start = 0.02
increment = 0.02
maximum = 0.2
# impulse variables
lengthMA = 34
lengthSignal = 9

In [13]:
# create a renko chart from the ohlc_dec_1 dataframe
r_full = Renko(ohlc, brick_size=brick_size)
# create a new dataframe from the renko features
renko_full_data = r_full.renko_df()

,open,high,low,close,volume,psar,psar_signal
datetime,,,,,,,
2024-02-13 18:24:41.746,1.0716,1.07190,1.07127,1.0719,364.0,1.072643,sell
2024-02-13 19:22:39.646,1.0716,1.07214,1.07130,1.0713,4691.0,1.072588,sell
2024-02-13 19:37:28.380,1.0716,1.07190,1.07122,1.0719,802.0,1.072535,sell
2024-02-13 20:22:08.406,1.0716,1.07201,1.07130,1.0713,2783.0,1.072456,sell
2024-02-13 20:51:46.414,1.0713,1.07138,1.07100,1.0710,1270.0,1.072382,sell
2024-02-13 21:01:07.195,1.0710,1.07100,1.07070,1.0707,460.0,1.072271,sell
2024-02-13 21:03:38.372,1.0707,1.07070,1.07040,1.0704,224.0,1.072114,sell
2024-02-13 21:58:23.352,1.0704,1.07068,1.07010,1.0701,3636.0,1.071909,sell
2024-02-13 22:25:40.512,1.0704,1.07070,1.07005,1.0707,1825.0,1.071655,sell


In [19]:
renko_full_data_psar_added = fx.psar_from_data(renko_full_data, increment, maximum)
renko_ip = fx.calc_impulse_macd(renko_full_data_psar_added, lengthMA, lengthSignal)
renko_ip['sma3'] = finta.TA.SMA(renko_ip, 3)
renko_ip.tail(10)

,open,high,low,close,volume,psar,psar_signal,hlc3,hi,lo,mi,md,sb,sh,sh_sign,impulse_signal,sma3
datetime,,,,,,,,,,,,,,,,,
2024-02-13 18:24:41.746,1.0716,1.07190,1.07127,1.0719,364.0,1.072643,sell,1.071690,1.072972,1.072422,1.071190,-0.001232,-0.001462,0.000231,1.0,buy,1.0716
2024-02-13 19:22:39.646,1.0716,1.07214,1.07130,1.0713,4691.0,1.072588,sell,1.071580,1.072947,1.072389,1.071199,-0.001190,-0.001400,0.000211,1.0,buy,1.0715
2024-02-13 19:37:28.380,1.0716,1.07190,1.07122,1.0719,802.0,1.072535,sell,1.071673,1.072916,1.072354,1.071218,-0.001136,-0.001344,0.000208,1.0,buy,1.0717
2024-02-13 20:22:08.406,1.0716,1.07201,1.07130,1.0713,2783.0,1.072456,sell,1.071537,1.072890,1.072323,1.071222,-0.001101,-0.001292,0.000191,1.0,buy,1.0715
2024-02-13 20:51:46.414,1.0713,1.07138,1.07100,1.0710,1270.0,1.072382,sell,1.071127,1.072845,1.072284,1.071181,-0.001103,-0.001244,0.000141,1.0,buy,1.0714
2024-02-13 21:01:07.195,1.0710,1.07100,1.07070,1.0707,460.0,1.072271,sell,1.070800,1.072791,1.072238,1.071108,-0.001130,-0.001207,0.000078,1.0,buy,1.0710
2024-02-13 21:03:38.372,1.0707,1.07070,1.07040,1.0704,224.0,1.072114,sell,1.070500,1.072730,1.072184,1.071009,-0.001175,-0.001185,0.000010,1.0,buy,1.0707
2024-02-13 21:58:23.352,1.0704,1.07068,1.07010,1.0701,3636.0,1.071909,sell,1.070293,1.072669,1.072122,1.070896,-0.001227,-0.001176,-0.000051,-1.0,sell,1.0704
2024-02-13 22:25:40.512,1.0704,1.07070,1.07005,1.0707,1825.0,1.071655,sell,1.070483,1.072611,1.072061,1.070815,-0.001247,-0.001171,-0.000076,-1.0,sell,1.0704
